# 모델 매개변수 최적화하기

## 기본(Pre-requisite) 코드

In [17]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [18]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


## 하이퍼파라미터(Hyperparameter)

In [19]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 손실 함수(loss function)

In [20]:
loss_fn = nn.CrossEntropyLoss()

## 옵티마이저(Optimizer)

In [21]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 전체 구현

In [22]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [23]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg_loss: {test_loss:>8f} \n")

In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')
    

Epoch 1
----------------------------
loss: 2.298429 [    0/60000]
loss: 2.283666 [ 6400/60000]
loss: 2.268619 [12800/60000]
loss: 2.270451 [19200/60000]
loss: 2.247594 [25600/60000]
loss: 2.215700 [32000/60000]
loss: 2.222404 [38400/60000]
loss: 2.187713 [44800/60000]
loss: 2.170295 [51200/60000]
loss: 2.152422 [57600/60000]
Test Error: 
 Accuracy: 45.8%, Avg_loss: 2.145942 

Epoch 2
----------------------------
loss: 2.151393 [    0/60000]
loss: 2.137103 [ 6400/60000]
loss: 2.081078 [12800/60000]
loss: 2.104055 [19200/60000]
loss: 2.048918 [25600/60000]
loss: 1.984249 [32000/60000]
loss: 2.010322 [38400/60000]
loss: 1.930340 [44800/60000]
loss: 1.913237 [51200/60000]
loss: 1.849964 [57600/60000]
Test Error: 
 Accuracy: 57.3%, Avg_loss: 1.855624 

Epoch 3
----------------------------
loss: 1.887823 [    0/60000]
loss: 1.853650 [ 6400/60000]
loss: 1.738228 [12800/60000]
loss: 1.779973 [19200/60000]
loss: 1.676607 [25600/60000]
loss: 1.622778 [32000/60000]
loss: 1.640519 [38400/60000]
lo